<a href="https://colab.research.google.com/github/navreen-waraich/EU-Energy-Transition-Monitor/blob/main/EU_Energy_Transition_Monitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EU Energy Transition Monitor 🌍⚡
**Analyzing the shift from Fossil Fuels to Renewables (2019-2024)**

*Author: Navreen Kaur Waraich*

In [ ]:
# 1. Install the necessary libraries
!pip install pandas duckdb seaborn matplotlib

import pandas as pd
import duckdb

# 2. Create a Virtual Database connection
con = duckdb.connect(database=':memory:')

print("Cloud Environment Ready!")

Cloud Environment Ready!


I am now loading the raw, detailed generation data from Ember, which is necessary because the raw data is often too large and unstructured for direct use. I will use Pandas and SQL (via DuckDB) to perform two critical steps: Filtering the scope to our 5 focus countries and the 2019–2024 period, and then Pivot/Aggregation to structure the data for Tableau.

In [ ]:
# 1. Load the dataset from the URL
url = "https://storage.googleapis.com/emb-prod-bkt-publicdata/public-downloads/europe_yearly_full_release_long_format.csv"
print("Downloading Data")

df = pd.read_csv(url)

print("Data Loaded Successfully!")
display(df.head())

print("\n Dataset Info:")
df.info()

Data Loaded Successfully!


,Area,ISO 3 code,Year,Area type,Continent,Ember region,EU,OECD,G20,G7,ASEAN,Category,Subcategory,Variable,Unit,Value,YoY absolute change,YoY % change
0,Austria,AUT,1990,Country or economy,Europe,Europe,1.0,1.0,0.0,0.0,0.0,Electricity demand,Demand,Demand,TWh,48.81,NaN,NaN
1,Austria,AUT,1990,Country or economy,Europe,Europe,1.0,1.0,0.0,0.0,0.0,Electricity demand,Demand per capita,Demand per capita,MWh,6.36,NaN,NaN
2,Austria,AUT,1990,Country or economy,Europe,Europe,1.0,1.0,0.0,0.0,0.0,Electricity generation,Aggregate fuel,Clean,%,66.25,NaN,NaN
3,Austria,AUT,1990,Country or economy,Europe,Europe,1.0,1.0,0.0,0.0,0.0,Electricity generation,Aggregate fuel,Coal,%,12.56,NaN,NaN
4,Austria,AUT,1990,Country or economy,Europe,Europe,1.0,1.0,0.0,0.0,0.0,Electricity generation,Aggregate fuel,Fossil,%,33.75,NaN,NaN



 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71414 entries, 0 to 71413
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Area                 71414 non-null  object 
 1   ISO 3 code           69409 non-null  object 
 2   Year                 71414 non-null  int64  
 3   Area type            71414 non-null  object 
 4   Continent            69409 non-null  object 
 5   Ember region         69409 non-null  object 
 6   EU                   69409 non-null  float64
 7   OECD                 69409 non-null  float64
 8   G20                  69409 non-null  float64
 9   G7                   69409 non-null  float64
 10  ASEAN                69409 non-null  float64
 11  Category             71414 non-null  object 
 12  Subcategory          71414 non-null  object 
 13  Variable             71414 non-null  object 
 14  Unit                 71414 non-null  object 
 15  Value               

**Data Pre-processing**: **Scope Definition**: I am now executing the data filtering to define our project scope precisely. I am selecting the five EU economies and restricting the analysis to the 2019–2024 period. This ensures the data is relevant, concise, and structured correctly for Tableau's analytical charting logic.

In [ ]:
# 1. Define the scope
focus_countries = [
    'Germany',
    'France',
    'Spain',
    'Netherlands',
    'Poland'
]
start_year = 2019
end_year = 2024

# 2. Filter the DataFrame using the defined scope
# .copy() ensures the filtered result is independent, preventing chained assignment warnings
df_filtered = df[
    (df['Year'] >= start_year) &
    (df['Year'] <= end_year) &
    (df['Area'].isin(focus_countries))
].copy()

print("Data Filtered Successfully for 2019-2024 and 5 Countries.")
display(df_filtered.head())

Data Filtered Successfully for 2019-2024 and 5 Countries.


,Area,ISO 3 code,Year,Area type,Continent,Ember region,EU,OECD,G20,G7,ASEAN,Category,Subcategory,Variable,Unit,Value,YoY absolute change,YoY % change
22459,France,FRA,2019,Country or economy,Europe,Europe,1.0,1.0,1.0,1.0,0.0,Electricity demand,Demand,Demand,TWh,507.85,-5.42,-1.06
22460,France,FRA,2019,Country or economy,Europe,Europe,1.0,1.0,1.0,1.0,0.0,Electricity demand,Demand per capita,Demand per capita,MWh,7.73,NaN,NaN
22461,France,FRA,2019,Country or economy,Europe,Europe,1.0,1.0,1.0,1.0,0.0,Electricity generation,Aggregate fuel,Clean,%,90.52,NaN,NaN
22462,France,FRA,2019,Country or economy,Europe,Europe,1.0,1.0,1.0,1.0,0.0,Electricity generation,Aggregate fuel,Coal,%,0.64,NaN,NaN
22463,France,FRA,2019,Country or economy,Europe,Europe,1.0,1.0,1.0,1.0,0.0,Electricity generation,Aggregate fuel,Fossil,%,9.48,NaN,NaN


**Data Validation and Re-execution**: Based on the comprehensive audit, I confirmed that the dataset contained non-TWh units and aggregated metrics like 'Demand' and 'Fossil' that were causing issues in the final pivot. I  am now running a revised, validated cleaning pipeline. This pipeline includes a critical filter on the unit and variable columns before all other steps, ensuring only clean TWh generation sources are processed.

In [ ]:
print("--- Comprehensive Audit of Critical Grouping & Measurement Columns ---")

# 1. Check all unique entries in grouping columns
print("\nUnique Variables (Granular Sources - To be Mapped):")
unique_variables = sorted(df_filtered['Variable'].unique())
print(unique_variables)

print("\nUnique Categories (Broad Groups - To Filter):")
unique_categories = sorted(df_filtered['Category'].unique())
print(unique_categories)

print("\nUnique Units (Measurement Check):")
unique_units = sorted(df_filtered['Unit'].unique())
print(unique_units)

print("\nUnique Areas (Countries):")
unique_areas = sorted(df_filtered['Area'].unique())
print(unique_areas)

print("\nUnique Years:")
unique_years = sorted(df_filtered['Year'].unique())
print(unique_years)

# 2. Check the range of the 'Value' column
print("\nValue Column Range (TWh Check):")
min_val = df_filtered['Value'].min()
max_val = df_filtered['Value'].max()
print(f"Minimum Value: {min_val}")
print(f"Maximum Value: {max_val}")

--- Comprehensive Audit of Critical Grouping & Measurement Columns ---

Unique Variables (Granular Sources - To be Mapped):
['Bioenergy', 'CO2 intensity', 'Clean', 'Coal', 'Demand', 'Demand per capita', 'Fossil', 'Gas', 'Hard coal', 'Hydro', 'Hydro, bioenergy and other renewables', 'Lignite', 'Net imports', 'Nuclear', 'Offshore wind', 'Onshore wind', 'Other fossil', 'Other renewables', 'Renewables', 'Solar', 'Total generation', 'Wind', 'Wind and solar']

Unique Categories (Broad Groups - To Filter):
['Electricity demand', 'Electricity generation', 'Electricity imports', 'Power sector emissions']

Unique Units (Measurement Check):
['%', 'MWh', 'MtCO2e', 'TWh', 'gCO2e per kWh']

Unique Areas (Countries):
['France', 'Germany', 'Netherlands', 'Poland', 'Spain']

Unique Years:
[np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]

Value Column Range (TWh Check):
Minimum Value: -89.85
Maximum Value: 771.66


**Data Quality and Standardization**: Before aggregation, I am executing several quality checks. Standardizing column names (snake_case) and checking for duplicates and missing values are essential to guarantee the final pivot table is reliable and the measures in Tableau are accurate.

In [ ]:
# Apply snake_case to column names
df_filtered.columns = df_filtered.columns.str.lower().str.replace(' ', '_')

print("Column Names Standardized.")
display(df_filtered.head(1))

Column Names Standardized.


,area,iso_3_code,year,area_type,continent,ember_region,eu,oecd,g20,g7,asean,category,subcategory,variable,unit,value,yoy_absolute_change,yoy_%_change
22459,France,FRA,2019,Country or economy,Europe,Europe,1.0,1.0,1.0,1.0,0.0,Electricity demand,Demand,Demand,TWh,507.85,-5.42,-1.06


**Refining Analytical Scope (Unit & Metric Filtering)** : To ensure a scientifically valid comparison of energy generation across countries, we must restrict the dataset to a single unit of measurement: Terawatt-hours (TWh). This eliminates incompatible metrics such as carbon emissions (MtCO2e) or efficiency percentages. Furthermore, I am explicitly removing aggregate variables (like 'Total generation' or 'Renewables') that represent sums of other rows. This prevents 'double-counting' when we perform our final aggregation, ensuring our total generation figures remain accurate and representative of individual energy sources.

In [ ]:
# --- CRITICAL FILTERING ---

# 1. Keep only TWh volume (Removes pollution, percentage, and intensity rows)
df_filtered = df_filtered[df_filtered['unit'] == 'TWh'].copy()

# 2. Define non-source metrics to exclude (the 'Total' and 'Aggregate' rows)
metrics_to_exclude = [
    'Demand', 'Demand per capita', 'Clean', 'Fossil', 'Renewables',
    'Wind and solar', 'Total generation', 'Net imports', 'CO2 intensity',
    'Hydro, bioenergy and other renewables'
]

# 3. Apply the filter
df_filtered = df_filtered[~df_filtered['variable'].isin(metrics_to_exclude)].copy()

# 4. HEALTH CHECK: Verify only TWh remains
remaining_units = df_filtered['unit'].unique()
print(f"Verified Units in Data: {remaining_units}")

if len(remaining_units) == 1 and remaining_units[0] == 'TWh':
    print(f"SUCCESS: Filter applied. {len(df_filtered)} rows of TWh data ready.")
else:
    print("ERROR: Filtering failed. Multiple units still present.")

Verified Units in Data: ['TWh']
SUCCESS: Filter applied. 378 rows of TWh data ready.


**Ensuring Record Uniqueness**: Data integrity requires that each observation is unique. In this step, I am enforcing a strict uniqueness constraint: for any given Year, Country (Area), and Energy Source (Variable), there must be exactly one value. Identifying and removing duplicate or conflicting records at this stage prevents inflated sums in our final visualizations and ensures the 'Granularity' of our data is perfectly consistent.

In [ ]:
# Check and Drop Duplicates
# We ensure there is only ONE value per Country per Year per Energy Source.
duplicate_count = df_filtered.duplicated(subset=['year', 'area', 'variable']).sum()

if duplicate_count > 0:
    # We keep 'first' to ensure we have a single unique record per category
    df_filtered = df_filtered.drop_duplicates(subset=['year', 'area', 'variable'], keep='first')
    print(f"Success: {duplicate_count} duplicate or conflicting rows removed.")
else:
    print("No duplicates found. Every (Year, Area, Variable) combination is unique.")

No duplicates found. Every (Year, Area, Variable) combination is unique.


**Data Quality**: **Physical Constraint Check**: I am now checking the TWh generation column (value) for any negative entries. In this dataset, negative values usually represent minor reporting adjustments or statistical anomalies, but they must be corrected to zero before we can aggregate the data accurately for analysis.

In [ ]:
# Correct negative TWh values to zero
negative_rows = (df_filtered['value'] < 0).sum()

if negative_rows > 0:
    # Safely modify the data using .loc
    df_filtered.loc[df_filtered['value'] < 0, 'value'] = 0
    print(f"Success: {negative_rows} negative values corrected to 0.")
else:
    print("No negative values found in the TWh generation data.")


No negative values found in the TWh generation data.


**Handling Data Sparsity**: I am performing a missing value audit. For energy generation data, if the value is missing, we cannot accurately impute it without external weather or grid data. Therefore, the most robust approach is to identify the volume of missing data and remove those records to maintain the integrity of our TWh sums.

In [ ]:
# Check Missing Values (Nulls)
# We focus on the 'value' column, as it's the core measurement.
missing_count = df_filtered['value'].isnull().sum()
total_rows = len(df_filtered)
missing_pct = (missing_count / total_rows) * 100

print(f"Audit Result: Found {missing_count} missing values ({missing_pct:.2f}% of data).")

if missing_count > 0:
    df_filtered = df_filtered.dropna(subset=['value']).copy()
    print("Action: Dropped rows with null values in 'value' column.")
else:
    print("Action: No missing values detected; data integrity confirmed.")

Audit Result: Found 0 missing values (0.00% of data).
Action: No missing values detected; data integrity confirmed.


**Statistical Validation**: To ensure data quality, I’m using a simple Z-score approach to identify outliers in the value column. While energy peaks are expected, extremely high values could indicate reporting errors or unit mismatches that would skew our Tableau visualizations.

In [ ]:
# Calculate statistics
mean_val = df_filtered['value'].mean()
std_val = df_filtered['value'].std()

# Define outlier threshold (3 standard deviations)
df_filtered['is_outlier'] = (df_filtered['value'] - mean_val).abs() > (3 * std_val)

outlier_count = df_filtered['is_outlier'].sum()
print(f"Audit Result: {outlier_count} statistical outliers identified and flagged.")

# Show the highest outliers for verification
if outlier_count > 0:
    display(df_filtered[df_filtered['is_outlier'] == True].sort_values(by='value', ascending=False).head(3))

Audit Result: 6 statistical outliers identified and flagged.


,area,iso_3_code,year,area_type,continent,ember_region,eu,oecd,g20,g7,asean,category,subcategory,variable,unit,value,yoy_absolute_change,yoy_%_change,is_outlier
22490,France,FRA,2019,Country or economy,Europe,Europe,1.0,1.0,1.0,1.0,0.0,Electricity generation,Fuel,Nuclear,TWh,399.01,-13.93,-3.37,True
22770,France,FRA,2024,Country or economy,Europe,Europe,1.0,1.0,1.0,1.0,0.0,Electricity generation,Fuel,Nuclear,TWh,380.45,42.25,12.49,True
22602,France,FRA,2021,Country or economy,Europe,Europe,1.0,1.0,1.0,1.0,0.0,Electricity generation,Fuel,Nuclear,TWh,379.36,25.53,7.22,True


**Standardizing Energy Sources & Categorization**: To prepare the data for visualization, I am standardizing the granular 'variable' names into clean energy categories. Additionally, I am creating an Energy Group dimension to classify sources as 'Renewable', 'Fossil', or 'Nuclear'. This multi-level categorization allows for both high-level summaries and granular source-by-source analysis in Tableau.

In [ ]:
# We use a dictionary that defines both the Clean Name and the Energy Group
# Structure: 'Original Name': ('Standard Name', 'Energy Group')
power_map = {
    'Hard coal': ('Coal', 'Fossil'),
    'Lignite': ('Coal', 'Fossil'),
    'Coal': ('Coal', 'Fossil'),
    'Gas': ('Gas', 'Fossil'),
    'Other fossil': ('Other Fossil', 'Fossil'),
    'Nuclear': ('Nuclear', 'Nuclear'),
    'Hydro': ('Hydro', 'Renewable'),
    'Wind': ('Wind', 'Renewable'),
    'Onshore wind': ('Wind', 'Renewable'),
    'Offshore wind': ('Wind', 'Renewable'),
    'Solar': ('Solar', 'Renewable'),
    'Bioenergy': ('Bioenergy', 'Renewable'),
    'Other renewables': ('Other Renewable', 'Renewable')
}

# 1. Map the 'variable' to our two new columns
df_filtered['source_standard'] = df_filtered['variable'].map(lambda x: power_map.get(x, ('Exclude', 'Exclude'))[0])
df_filtered['energy_group'] = df_filtered['variable'].map(lambda x: power_map.get(x, ('Exclude', 'Exclude'))[1])

# 2. Final safety filter to remove anything marked as 'Exclude'
df_filtered = df_filtered[df_filtered['source_standard'] != 'Exclude'].copy()

print(f"Mapping Complete. Current Data Shape: {df_filtered.shape}")
print("\nUnique Groups Created:", df_filtered['energy_group'].unique())

# Quick look at the result
display(df_filtered[['variable', 'source_standard', 'energy_group']].drop_duplicates().sort_values('energy_group'))

Mapping Complete. Current Data Shape: (378, 21)

Unique Groups Created: ['Fossil' 'Renewable' 'Nuclear']


,variable,source_standard,energy_group
22469,Coal,Coal,Fossil
22486,Gas,Gas,Fossil
22487,Hard coal,Coal,Fossil
22489,Lignite,Coal,Fossil
22492,Other fossil,Other Fossil,Fossil
22490,Nuclear,Nuclear,Nuclear
22473,Wind,Wind,Renewable
22485,Bioenergy,Bioenergy,Renewable
22488,Hydro,Hydro,Renewable
22491,Onshore wind,Wind,Renewable


**Data Preparation**: **Final Aggregation (Pivoting)**: To prepare the final dataset for Tableau, I am using DuckDB to execute a SQL-based pivot. This approach allows for efficient aggregation and reshaping of the data. By transforming the 'long' format into a 'wide' format—where each energy source is its own column—we create a structure that is highly optimized for stacked area charts and year-over-year comparisons.

In [ ]:
# 1. Re-register the latest version of your filtered data
con.register('df_final_clean', df_filtered)

# 2. Advanced SQL Pivot
# This creates a "Wide" format: 1 row per Country per Year
query = """
SELECT
    year,
    area,
    SUM(CASE WHEN source_standard = 'Coal' THEN value ELSE 0 END) AS coal_twh,
    SUM(CASE WHEN source_standard = 'Gas' THEN value ELSE 0 END) AS gas_twh,
    SUM(CASE WHEN source_standard = 'Nuclear' THEN value ELSE 0 END) AS nuclear_twh,
    SUM(CASE WHEN source_standard = 'Hydro' THEN value ELSE 0 END) AS hydro_twh,
    SUM(CASE WHEN source_standard = 'Wind' THEN value ELSE 0 END) AS wind_twh,
    SUM(CASE WHEN source_standard = 'Solar' THEN value ELSE 0 END) AS solar_twh,
    SUM(CASE WHEN source_standard = 'Bioenergy' THEN value ELSE 0 END) AS bioenergy_twh,
    SUM(CASE WHEN source_standard = 'Other Fossil' THEN value ELSE 0 END) AS other_fossil_twh,
    SUM(CASE WHEN source_standard = 'Other Renewable' THEN value ELSE 0 END) AS other_renewable_twh,
    -- Also include a flag if ANY row in that year was an outlier
    MAX(CASE WHEN is_outlier = True THEN 1 ELSE 0 END) AS contains_outlier
FROM df_final_clean
GROUP BY year, area
ORDER BY area, year
"""

# 3. Execute and store
df_tableau = con.execute(query).fetchdf()

print("SQL Pivot Successful: Data is now in 'Wide' format for Tableau.")
display(df_tableau.head())

SQL Pivot Successful: Data is now in 'Wide' format for Tableau.


,year,area,coal_twh,gas_twh,nuclear_twh,hydro_twh,wind_twh,solar_twh,bioenergy_twh,other_fossil_twh,other_renewable_twh,contains_outlier
0,2019,France,7.28,39.24,399.01,56.91,69.44,12.02,8.64,10.73,0.61,1
1,2020,France,6.18,35.25,353.83,62.52,80.10,13.09,8.76,10.05,0.61,1
2,2021,France,10.88,33.31,379.36,59.54,74.24,15.28,9.58,10.10,0.58,1
3,2022,France,8.32,45.74,294.73,45.56,76.00,19.34,9.67,10.45,0.60,1
4,2023,France,3.22,30.02,338.20,55.90,96.10,22.59,10.48,9.43,0.58,1


**Engineering Core KPI Columns** : To streamline the visualization process and ensure consistent metric definitions, I am pre-calculating core KPIs within the Python environment. By adding 'Total Generation' and 'Renewable Share %' directly to our final dataset, we create a 'Gold Layer' table. This practice ensures that any stakeholder using this data—whether in Tableau, PowerBI, or Excel—will see the exact same verified figures.

In [ ]:
# 1. Calculate Total Generation (Sum of all fuel columns)
fuel_columns = [
    'coal_twh', 'gas_twh', 'nuclear_twh', 'hydro_twh',
    'wind_twh', 'solar_twh', 'bioenergy_twh',
    'other_fossil_twh', 'other_renewable_twh'
]
df_tableau['total_generation_twh'] = df_tableau[fuel_columns].sum(axis=1)

# 2. Calculate Renewable Total
renewable_cols = ['hydro_twh', 'wind_twh', 'solar_twh', 'bioenergy_twh', 'other_renewable_twh']
df_tableau['renewable_total_twh'] = df_tableau[renewable_cols].sum(axis=1)

# 3. Calculate Renewable Share %
# We use a small epsilon (1e-9) to avoid division by zero errors
df_tableau['renewable_share_pct'] = (df_tableau['renewable_total_twh'] / (df_tableau['total_generation_twh'] + 1e-9)) * 100

print("KPI Columns Added Successfully:")
display(df_tableau[['area', 'year', 'total_generation_twh', 'renewable_share_pct']].head())

KPI Columns Added Successfully:


,area,year,total_generation_twh,renewable_share_pct
0,France,2019,603.88,24.445254
1,France,2020,570.39,28.941601
2,France,2021,592.87,26.855803
3,France,2022,510.41,29.617366
4,France,2023,566.52,32.770246


In [ ]:
# Save the dataset
df_tableau.to_csv('european_energy_transition.csv', index=False)

print("SUCCESS: Final Dataset Exported!")
print(f"Final File: 'european_energy_transition_final.csv'")
print(f"Final Shape: {df_tableau.shape} (Rows, Columns)")

SUCCESS: Final Dataset Exported!
Final File: 'european_energy_transition_final.csv'
Final Shape: (30, 15) (Rows, Columns)
